<a href="https://colab.research.google.com/github/2yeonsong/MachineLearning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 트리의 앙상블

In [2]:
# 랜덤 포레스트
# 앙상블 학습 중 가장 유명하고 안정적인 성능을 제공
# 어떻게 결정 트리를 랜덤하게 만들것인가가 핵심

In [3]:
# 훈련 세트에 대해서 깊이(세분화) 훈련 하면 과대적합이 생긴다.? 근데...
# 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합 되는 것을 막아주고
# 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음

# 랜덤 포레스트는 누락된 값이 있어도 처리가 가능하다는 장점이 있다.

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [6]:
# 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [7]:
# oob_score_
# 부트스트랩 샘플에 포함되지 않는 샘플
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [8]:
# 엑스트라트리

In [9]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [12]:
# 그레디언트 부스팅
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
# 과대 적합에 강하다(일반화 성능을 기대할 수 있다)

In [13]:
# 부스팅이므로 예측 모델을 순차적으로 더해나간 후 최종 모델을 예측
# 즉, 이전 모델의 오류(실제값과 예측값의 차이)를 새로운 모델을 추가하면서 줄이는 것
# 사진 있음

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [ ]:
# 일반적으로 랜덤 포레스트보다 성능이 좋다.
# 하지만 훈련 속도가 더 느리다, CPU 한개만 쓴다.

In [22]:
# 히스토그램 기반 그레이디언 부스팅
# 정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘
# 과대적합을 억제하면서 그레이디언트보다 더 높은 성능을 보여준다

In [18]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [19]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [23]:
# 테스트 세트에 대한 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [24]:
hgb.score(test_input, test_target)

0.8723076923076923

In [25]:
# 사이킷런이 아닌 다른 곳에서 그레이디언트 부스팅 알고리즘을 구현한 라이브러리

In [27]:
# XGBoost

In [28]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [29]:
# LightGBM

In [30]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
